## 스마트 정수장 알고리즘 - 약품 공정 - EDA

In [ ]:
import numpy as np
import pandas as pd
import datetime
import scipy
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import r_regression
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import pickle
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import platform

if platform.system() == "Darwin":  #
    plt.rc("font", family="AppleGothic")
else:
    plt.rc("font", family="NanumGothic")

fe = fm.FontEntry(
    fname=r"/usr/share/fonts/truetype/nanum/NanumGothic.ttf",  # ttf 파일이 저장되어 있는 경로
    name="NanumGothic",
)  # 원하는 폰트 설정
fm.fontManager.ttflist.insert(0, fe)  # Matplotlib에 폰트 추가

plt.rcParams.update({"font.size": 18, "font.family": "NanumGothic"})  # 폰트 설정

plt.rcParams["axes.unicode_minus"] = False

---
### 문제 정의
> 데이터 분석을 수행하기 위해서는 해결하려는 문제가 무엇인지 정의되어야 한다. </br>
> 분석하고자 하는 분야를 이해하고 구체적으로 문제를 정의한다.

**약품 공정의 목적** </br>
응집$\cdot$침전 공정은 여과 공정 이전에 일차적으로 탁질을 제거하여 응집지의 폐색을 늦추고 안정적인 운영을 가능하도록 한다. </br>
약품 공정에서는 응집$\cdot$침전 공정이 효과적으로 작동할 수 있도록 적절한 응집제 및 알칼리제를 주입한다. </br>
약품 공정이 잘 작동되었는지를 확인하려면 침전지 탁도가 안정적으로 유지되는지로 판단할 수 있다.

**약품 공정 AI 알고리즘의 목표** </br>
근무자의 결정을 모사하여 적절한 응집제 주입률을 구하는 것이 목표이다.

---
### 1.2. 데이터 정의
> 앞서 정의한 문제에 의거하여 필요한 데이터를 정의하고, 해당 데이터를 수집하는 단계이다.

#### 1.2.1. 필요 데이터

근무자의 결정을 모사하는 것이 목표이기 때문에, 근무자의 주입률 결정 방법을 이해해야 한다. </br>
기존에 사용되던 근무자의 결정 방법은 다음과 같다. </br>

**응집제 주입률 결정 방법** </br>
- 탁도에 따라 조견표로 응집제 주입률 결정 </br>
- 탁도, pH, 알칼리도와 같은 수질 인자로부터 경험적으로 응집제 주입률 결정 </br>
- 시네드라와 같은 조류 발생 시 여과지 폐색을 줄이기 위해 강화 응집 </br>
- 청소나 점검과 같은 이유로 일부 침전지, 여과지를 사용하지 않을 시 강화 응집 </br>
- 여름철 태풍과 같이 강우가 내릴 때 강화 응집 </br>

이때 강우가 내리는 것은 원수의 탁도, 알칼리도 등으로 확인할 수 있다.

따라서 필요한 데이터는 다음과 같다.</br>

**필요 데이터**
- 탁도, pH, 알칼리도, 전기전도도, 수온, 망간과 같은 원수 수질 인자
- 사용하고 있는 침전지 지수와 여과지 지수, 체류시간
- 시네드라의 발생 여부 및 발생 조건
- 응집제 주입률
- 침전지 탁도

#### 1.2.2. 수집 데이터

In [ ]:
!unzip SN_2020.zip

In [ ]:
file = "SN_2020.csv"
load_df = pd.read_csv(file)
load_df.head(5)

In [ ]:
load_df = load_df.set_index("시간")
load_df.index = pd.DatetimeIndex(load_df.index)
load_df.head(5)

In [ ]:
cols = [
    "원수 탁도",
    "원수 pH",
    "원수 알칼리도",
    "원수 전기전도도",
    "원수 수온",
    "3단계 원수 유입 유량",
    "3단계 1계열 응집제 주입률",
    "3단계 침전지 탁도",
]
raw_df = load_df[cols]
raw_df.head(5)

---
### 1.3. 탐색적 데이터 분석 (EDA)

#### 1.3.1. 통계값 확인

In [ ]:
raw_df.describe()

In [ ]:
mean = raw_df.mean()
mean

In [ ]:
std = raw_df.std()
std

#### 1.3.2. 시계열 차트

In [ ]:
fig = plt.figure(figsize=(16, 5))
plt.plot(raw_df.index, raw_df["원수 탁도"], label="원수 탁도")
plt.axhline(mean["원수 탁도"], c="r", label="평균")
plt.axhspan(
    mean["원수 탁도"] - std["원수 탁도"],
    mean["원수 탁도"] + std["원수 탁도"],
    color="grey",
    alpha=0.2,
    label="1 표준편차 구역",
)
plt.ylabel("원수 탁도")
plt.grid()
plt.legend()

In [ ]:
col = "원수 pH"
fig = plt.figure(figsize=(16, 5))
plt.plot(raw_df.index, raw_df[col], label=col)
plt.axhline(mean[col], c="r", label="평균")
plt.axhspan(
    mean[col] - std[col],
    mean[col] + std[col],
    color="grey",
    alpha=0.2,
    label="1 표준편차 구역",
)
plt.ylabel(col)
plt.grid()
plt.legend()

In [ ]:
col = "원수 알칼리도"
fig = plt.figure(figsize=(16, 5))
plt.plot(raw_df.index, raw_df[col], label=col)
plt.axhline(mean[col], c="r", label="평균")
plt.axhspan(
    mean[col] - std[col],
    mean[col] + std[col],
    color="grey",
    alpha=0.2,
    label="1 표준편차 구역",
)
plt.ylabel(col)
plt.grid()
plt.legend()

In [ ]:
col = "원수 전기전도도"
fig = plt.figure(figsize=(16, 5))
plt.plot(raw_df.index, raw_df[col], label=col)
plt.axhline(mean[col], c="r", label="평균")
plt.axhspan(
    mean[col] - std[col],
    mean[col] + std[col],
    color="grey",
    alpha=0.2,
    label="1 표준편차 구역",
)
plt.ylabel(col)
plt.grid()
plt.legend()

In [ ]:
col = "원수 수온"
fig = plt.figure(figsize=(16, 5))
plt.plot(raw_df.index, raw_df[col], label=col)
plt.axhline(mean[col], c="r", label="평균")
plt.axhspan(
    mean[col] - std[col],
    mean[col] + std[col],
    color="grey",
    alpha=0.2,
    label="1 표준편차 구역",
)
plt.ylabel(col)
plt.grid()
plt.legend()

In [ ]:
col = "3단계 1계열 응집제 주입률"
fig = plt.figure(figsize=(16, 5))
plt.plot(raw_df.index, raw_df[col], label=col)
plt.axhline(mean[col], c="r", label="평균")
plt.axhspan(
    mean[col] - std[col],
    mean[col] + std[col],
    color="grey",
    alpha=0.2,
    label="1 표준편차 구역",
)
plt.ylabel(col)
plt.grid()
plt.legend()

In [ ]:
col = "3단계 침전지 탁도"
fig = plt.figure(figsize=(16, 5))
plt.plot(raw_df.index, raw_df[col], label=col)
plt.axhline(mean[col], c="r", label="평균")
plt.axhspan(
    mean[col] - std[col],
    mean[col] + std[col],
    color="grey",
    alpha=0.2,
    label="1 표준편차 구역",
)
plt.ylabel(col)
plt.grid()
plt.legend()

#### 1.3.3. 히스토그램

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(raw_df["원수 탁도"], bins=30, facecolor="grey", edgecolor="k", alpha=0.5)
plt.yscale("log")
plt.grid()
plt.xlabel("원수 탁도")
plt.ylabel("데이터 개수")

In [ ]:
col = "원수 pH"
plt.figure(figsize=(8, 6))
plt.hist(raw_df[col], bins=30, facecolor="grey", edgecolor="k", alpha=0.5)
plt.yscale("log")
plt.grid()
plt.xlabel(col)
plt.ylabel("데이터 개수")

In [ ]:
col = "원수 알칼리도"
plt.figure(figsize=(8, 6))
plt.hist(raw_df[col], bins=30, facecolor="grey", edgecolor="k", alpha=0.5)
plt.yscale("log")
plt.grid()
plt.xlabel(col)
plt.ylabel("데이터 개수")

In [ ]:
col = "원수 전기전도도"
plt.figure(figsize=(8, 6))
plt.hist(raw_df[col], bins=30, facecolor="grey", edgecolor="k", alpha=0.5)
plt.yscale("log")
plt.grid()
plt.xlabel(col)
plt.ylabel("데이터 개수")

In [ ]:
col = "원수 수온"
plt.figure(figsize=(8, 6))
plt.hist(raw_df[col], bins=30, facecolor="grey", edgecolor="k", alpha=0.5)
plt.yscale("log")
plt.grid()
plt.xlabel(col)
plt.ylabel("데이터 개수")

In [ ]:
col = "3단계 1계열 응집제 주입률"
plt.figure(figsize=(8, 6))
plt.hist(raw_df[col], bins=30, facecolor="grey", edgecolor="k", alpha=0.5)
plt.yscale("log")
plt.grid()
plt.xlabel(col)
plt.ylabel("데이터 개수")

In [ ]:
col = "3단계 침전지 탁도"
plt.figure(figsize=(8, 6))
plt.hist(raw_df[col], bins=30, facecolor="grey", edgecolor="k", alpha=0.5)
plt.yscale("log")
plt.grid()
plt.xlabel(col)
plt.ylabel("데이터 개수")

#### 1.3.2. 분산형 차트

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(raw_df["원수 탁도"], raw_df["3단계 1계열 응집제 주입률"], s=3)
plt.grid()
plt.xlabel("원수 탁도")
plt.ylabel("응집제 주입률")

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(raw_df["원수 탁도"], raw_df["3단계 침전지 탁도"], s=3)
plt.grid()
plt.xlabel("원수 탁도")
plt.ylabel("3단계 침전지 탁도")

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(raw_df["원수 탁도"], raw_df["원수 pH"], s=3)
plt.grid()
plt.xlabel("원수 탁도")
plt.ylabel("원수 pH")

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(raw_df["원수 pH"], raw_df["3단계 1계열 응집제 주입률"], s=3)
plt.grid()
plt.xlabel("원수 pH")
plt.ylabel("3단계 1계열 응집제 주입률")

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(raw_df["원수 탁도"], raw_df["3단계 침전지 탁도"], s=3)
plt.grid()
plt.xlabel("원수 탁도")
plt.ylabel("3단계 침전지 탁도")